In [253]:
# !pip install finta

In [254]:
import pandas as pd
from finta import TA
from datetime import timedelta

class StockDataProcessor:
    """
    A class for processing stock data, calculating technical indicators, crossovers, and days since crossovers.

    Attributes:
    df (pd.DataFrame): The DataFrame containing stock data and calculated indicators.
    """
    def __init__(self, df):
        """
        Initialize the StockDataProcessor with a DataFrame containing stock data.

        Args:
        df (pd.DataFrame): DataFrame with 'Close' prices as the index.

        Attributes:
        df (pd.DataFrame): The DataFrame containing stock data and calculated indicators.
        """
        self.df = df
        self.calculate_indicators()
        self.calculate_days_since_crossover()

    def calculate_indicators(self):
        """
        Calculate SMA5, SMA20, EMA5, and EMA20 using the finta library for the provided DataFrame and add crossover columns.

        Args: None

        Returns: None
        """
        self.df['SMA5'] = TA.SMA(self.df, 5)
        self.df['SMA20'] = TA.SMA(self.df, 20)
        self.df['EMA5'] = TA.EMA(self.df, 5)
        self.df['EMA20'] = TA.EMA(self.df, 20)

        # Calculate crossovers
        self.df['Close_SMA5_Crossover'] = self.df['Close'] // self.df['SMA5']
        self.df['Close_SMA20_Crossover'] = self.df['Close'] // self.df['SMA20']
        self.df['SMA5_SMA20_Crossover'] = self.df['SMA5'] // self.df['SMA20']
        self.df['Close_EMA5_Crossover'] = self.df['Close'] // self.df['EMA5']
        self.df['Close_EMA20_Crossover'] = self.df['Close'] // self.df['EMA20']
        self.df['EMA5_EMA20_Crossover'] = self.df['EMA5'] // self.df['EMA20']
        self.df['SMA5_EMA5_Crossover'] = self.df['SMA5'] // self.df['EMA5']
        self.df['SMA5_EMA20_Crossover'] = self.df['SMA5'] // self.df['EMA20']
        self.df['SMA20_EMA5_Crossover'] = self.df['SMA20'] // self.df['EMA5']
        self.df['SMA20_EMA20_Crossover'] = self.df['SMA20'] // self.df['EMA20']

        # Determine crossover directions (up = 1, down = -1, if crossover doesn't happen, 0)
        crossover_columns = [
            'Close_SMA5_Crossover',
            'Close_SMA20_Crossover',
            'SMA5_SMA20_Crossover',
            'Close_EMA5_Crossover',
            'Close_EMA20_Crossover',
            'EMA5_EMA20_Crossover',
            'SMA5_EMA5_Crossover',
            'SMA5_EMA20_Crossover',
            'SMA20_EMA5_Crossover',
            'SMA20_EMA20_Crossover'
        ]

        for col in crossover_columns:
            self.df[col + "_Direction"] = self.df[col].diff().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

    def calculate_days_since_crossover(self):
      """
        Calculate the number of days since crossovers for different indicator combinations and update the DataFrame.

        Args: None

        Returns: None
        """
      # Initialize a dictionary to keep track of the latest crossover dates and day counts
      last_crossover_dates = {}
      day_counts = {}

      crossover_columns = [
          'Close_SMA5_Crossover',
          'Close_SMA20_Crossover',
          'SMA5_SMA20_Crossover',
          'Close_EMA5_Crossover',
          'Close_EMA20_Crossover',
          'EMA5_EMA20_Crossover',
          'SMA5_EMA5_Crossover',
          'SMA5_EMA20_Crossover',
          'SMA20_EMA5_Crossover',
          'SMA20_EMA20_Crossover'
      ]

      # Loop through the crossover columns and calculate the number of days since crossover using an increment-based approach
      for col in crossover_columns:
          self.df[f'Days_Since_{col}'] = 0  # Initialize the days since crossover column
          last_crossover_dates[col] = None
          day_counts[col] = 0

      for i, row in self.df.iterrows():
          for col in crossover_columns:
              if col in self.df.columns:
                  if row[col] == 1:  # Crossover occurred
                      last_crossover_dates[col] = i
                      day_counts[col] = 0  # Reset the day count to 0 when a new crossover occurs
                  elif last_crossover_dates[col] is not None:
                      day_counts[col] += 1
                      self.df.at[i, f'Days_Since_{col}'] = day_counts[col]

    def get_processed_data(self):
        """
        Get the DataFrame with calculated indicators, crossovers, directions, and days since crossover.

        Args:
        None

        Returns:
        pd.DataFrame: Processed DataFrame.
        """
        return self.df

In [255]:
df = pd.read_csv("/content/tickers.csv")
df = df[df.tic == "^TNX"]
df = df.drop(["Unnamed: 0", "tic", "day"], axis=1)
df = df.rename(columns = {'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df.sort_values('date', ascending=True, inplace=True)

In [256]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [257]:
processor = StockDataProcessor(df)
processed_df = processor.get_processed_data()
processed_df.fillna(0, inplace=True)
processed_df.head()

,Open,High,Low,Close,Volume,SMA5,SMA20,EMA5,EMA20,Close_SMA5_Crossover,Close_SMA20_Crossover,SMA5_SMA20_Crossover,Close_EMA5_Crossover,Close_EMA20_Crossover,EMA5_EMA20_Crossover,SMA5_EMA5_Crossover,SMA5_EMA20_Crossover,SMA20_EMA5_Crossover,SMA20_EMA20_Crossover,Close_SMA5_Crossover_Direction,Close_SMA20_Crossover_Direction,SMA5_SMA20_Crossover_Direction,Close_EMA5_Crossover_Direction,Close_EMA20_Crossover_Direction,EMA5_EMA20_Crossover_Direction,SMA5_EMA5_Crossover_Direction,SMA5_EMA20_Crossover_Direction,SMA20_EMA5_Crossover_Direction,SMA20_EMA20_Crossover_Direction,Days_Since_Close_SMA5_Crossover,Days_Since_Close_SMA20_Crossover,Days_Since_SMA5_SMA20_Crossover,Days_Since_Close_EMA5_Crossover,Days_Since_Close_EMA20_Crossover,Days_Since_EMA5_EMA20_Crossover,Days_Since_SMA5_EMA5_Crossover,Days_Since_SMA5_EMA20_Crossover,Days_Since_SMA20_EMA5_Crossover,Days_Since_SMA20_EMA20_Crossover
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-01-04,6.530,6.548,6.485,6.485,0,0.0000,0.0,6.485000,6.485000,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2000-01-05,6.521,6.599,6.508,6.599,0,0.0000,0.0,6.553400,6.544850,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2000-01-06,6.558,6.585,6.540,6.549,0,0.0000,0.0,6.551316,6.546374,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2000-01-07,6.545,6.595,6.504,6.504,0,0.0000,0.0,6.531662,6.534141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,2,1,1,0,0,0,0
2000-01-11,6.600,6.664,6.595,6.664,0,6.5602,0.0,6.582464,6.565553,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [258]:
# Define the desired column order
desired_order = [
    'Open',
    'High',
    'Low',
    'Close',
    'Volume',
    'SMA5',
    'SMA20',
    'EMA5',
    'EMA20',
    'Close_SMA5_Crossover',
    'Close_SMA5_Crossover_Direction',
    'Days_Since_Close_SMA5_Crossover',
    'Close_SMA20_Crossover',
    'Close_SMA20_Crossover_Direction',
    'Days_Since_Close_SMA20_Crossover',
    'SMA5_SMA20_Crossover',
    'SMA5_SMA20_Crossover_Direction',
    'Days_Since_SMA5_SMA20_Crossover',
    'Close_EMA5_Crossover',
    'Close_EMA5_Crossover_Direction',
    'Days_Since_Close_EMA5_Crossover',
    'Close_EMA20_Crossover',
    'Close_EMA20_Crossover_Direction',
    'Days_Since_Close_EMA20_Crossover',
    'EMA5_EMA20_Crossover',
    'EMA5_EMA20_Crossover_Direction',
    'Days_Since_EMA5_EMA20_Crossover',
    'SMA5_EMA5_Crossover',
    'SMA5_EMA5_Crossover_Direction',
    'Days_Since_SMA5_EMA5_Crossover',
    'SMA5_EMA20_Crossover',
    'SMA5_EMA20_Crossover_Direction',
    'Days_Since_SMA5_EMA20_Crossover',
    'SMA20_EMA5_Crossover',
    'SMA20_EMA5_Crossover_Direction',
    'Days_Since_SMA20_EMA5_Crossover',
    'SMA20_EMA20_Crossover',
    'SMA20_EMA20_Crossover_Direction',
    'Days_Since_SMA20_EMA20_Crossover',
]

# Reorder the columns
processed_df = processed_df[desired_order]
processed_df.head(10)

,Open,High,Low,Close,Volume,SMA5,SMA20,EMA5,EMA20,Close_SMA5_Crossover,Close_SMA5_Crossover_Direction,Days_Since_Close_SMA5_Crossover,Close_SMA20_Crossover,Close_SMA20_Crossover_Direction,Days_Since_Close_SMA20_Crossover,SMA5_SMA20_Crossover,SMA5_SMA20_Crossover_Direction,Days_Since_SMA5_SMA20_Crossover,Close_EMA5_Crossover,Close_EMA5_Crossover_Direction,Days_Since_Close_EMA5_Crossover,Close_EMA20_Crossover,Close_EMA20_Crossover_Direction,Days_Since_Close_EMA20_Crossover,EMA5_EMA20_Crossover,EMA5_EMA20_Crossover_Direction,Days_Since_EMA5_EMA20_Crossover,SMA5_EMA5_Crossover,SMA5_EMA5_Crossover_Direction,Days_Since_SMA5_EMA5_Crossover,SMA5_EMA20_Crossover,SMA5_EMA20_Crossover_Direction,Days_Since_SMA5_EMA20_Crossover,SMA20_EMA5_Crossover,SMA20_EMA5_Crossover_Direction,Days_Since_SMA20_EMA5_Crossover,SMA20_EMA20_Crossover,SMA20_EMA20_Crossover_Direction,Days_Since_SMA20_EMA20_Crossover
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-01-04,6.530,6.548,6.485,6.485,0,0.0000,0.0,6.485000,6.485000,0.0,0,0,0.0,0,0,0.0,0,0,1.0,0,0,1.0,0,0,1.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0
2000-01-05,6.521,6.599,6.508,6.599,0,0.0000,0.0,6.553400,6.544850,0.0,0,0,0.0,0,0,0.0,0,0,1.0,0,0,1.0,0,0,1.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0
2000-01-06,6.558,6.585,6.540,6.549,0,0.0000,0.0,6.551316,6.546374,0.0,0,0,0.0,0,0,0.0,0,0,0.0,-1,1,1.0,0,0,1.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0
2000-01-07,6.545,6.595,6.504,6.504,0,0.0000,0.0,6.531662,6.534141,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,2,0.0,-1,1,0.0,-1,1,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0
2000-01-11,6.600,6.664,6.595,6.664,0,6.5602,0.0,6.582464,6.565553,1.0,0,0,0.0,0,0,0.0,0,0,1.0,1,0,1.0,1,0,1.0,1,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0
2000-01-12,6.659,6.696,6.645,6.696,0,6.6024,0.0,6.623952,6.593071,1.0,0,0,0.0,0,0,0.0,0,0,1.0,0,0,1.0,0,0,1.0,0,0,0.0,0,0,1.0,1,0,0.0,0,0,0.0,0,0
2000-01-13,6.664,6.705,6.618,6.618,0,6.6062,0.0,6.621845,6.597785,1.0,0,0,0.0,0,0,0.0,0,0,0.0,-1,1,1.0,0,0,1.0,0,0,0.0,0,0,1.0,0,0,0.0,0,0,0.0,0,0
2000-01-14,6.623,6.688,6.563,6.674,0,6.6312,0.0,6.639936,6.610959,1.0,0,0,0.0,0,0,0.0,0,0,1.0,1,0,1.0,0,0,1.0,0,0,0.0,0,0,1.0,0,0,0.0,0,0,0.0,0,0
2000-01-18,6.706,6.757,6.702,6.748,0,6.6800,0.0,6.676919,6.632941,1.0,0,0,0.0,0,0,0.0,0,0,1.0,0,0,1.0,0,0,1.0,0,0,1.0,1,0,1.0,0,0,0.0,0,0,0.0,0,0


In [259]:
processed_df.head(100)

,Open,High,Low,Close,Volume,SMA5,SMA20,EMA5,EMA20,Close_SMA5_Crossover,Close_SMA5_Crossover_Direction,Days_Since_Close_SMA5_Crossover,Close_SMA20_Crossover,Close_SMA20_Crossover_Direction,Days_Since_Close_SMA20_Crossover,SMA5_SMA20_Crossover,SMA5_SMA20_Crossover_Direction,Days_Since_SMA5_SMA20_Crossover,Close_EMA5_Crossover,Close_EMA5_Crossover_Direction,Days_Since_Close_EMA5_Crossover,Close_EMA20_Crossover,Close_EMA20_Crossover_Direction,Days_Since_Close_EMA20_Crossover,EMA5_EMA20_Crossover,EMA5_EMA20_Crossover_Direction,Days_Since_EMA5_EMA20_Crossover,SMA5_EMA5_Crossover,SMA5_EMA5_Crossover_Direction,Days_Since_SMA5_EMA5_Crossover,SMA5_EMA20_Crossover,SMA5_EMA20_Crossover_Direction,Days_Since_SMA5_EMA20_Crossover,SMA20_EMA5_Crossover,SMA20_EMA5_Crossover_Direction,Days_Since_SMA20_EMA5_Crossover,SMA20_EMA20_Crossover,SMA20_EMA20_Crossover_Direction,Days_Since_SMA20_EMA20_Crossover
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-01-04,6.530,6.548,6.485,6.485,0,0.0000,0.00000,6.485000,6.485000,0.0,0,0,0.0,0,0,0.0,0,0,1.0,0,0,1.0,0,0,1.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0
2000-01-05,6.521,6.599,6.508,6.599,0,0.0000,0.00000,6.553400,6.544850,0.0,0,0,0.0,0,0,0.0,0,0,1.0,0,0,1.0,0,0,1.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0
2000-01-06,6.558,6.585,6.540,6.549,0,0.0000,0.00000,6.551316,6.546374,0.0,0,0,0.0,0,0,0.0,0,0,0.0,-1,1,1.0,0,0,1.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0
2000-01-07,6.545,6.595,6.504,6.504,0,0.0000,0.00000,6.531662,6.534141,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,2,0.0,-1,1,0.0,-1,1,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0
2000-01-11,6.600,6.664,6.595,6.664,0,6.5602,0.00000,6.582464,6.565553,1.0,0,0,0.0,0,0,0.0,0,0,1.0,1,0,1.0,1,0,1.0,1,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0
2000-01-12,6.659,6.696,6.645,6.696,0,6.6024,0.00000,6.623952,6.593071,1.0,0,0,0.0,0,0,0.0,0,0,1.0,0,0,1.0,0,0,1.0,0,0,0.0,0,0,1.0,1,0,0.0,0,0,0.0,0,0
2000-01-13,6.664,6.705,6.618,6.618,0,6.6062,0.00000,6.621845,6.597785,1.0,0,0,0.0,0,0,0.0,0,0,0.0,-1,1,1.0,0,0,1.0,0,0,0.0,0,0,1.0,0,0,0.0,0,0,0.0,0,0
2000-01-14,6.623,6.688,6.563,6.674,0,6.6312,0.00000,6.639936,6.610959,1.0,0,0,0.0,0,0,0.0,0,0,1.0,1,0,1.0,0,0,1.0,0,0,0.0,0,0,1.0,0,0,0.0,0,0,0.0,0,0
2000-01-18,6.706,6.757,6.702,6.748,0,6.6800,0.00000,6.676919,6.632941,1.0,0,0,0.0,0,0,0.0,0,0,1.0,0,0,1.0,0,0,1.0,0,0,1.0,1,0,1.0,0,0,0.0,0,0,0.0,0,0


In [260]:
processed_df.describe()

,Open,High,Low,Close,Volume,SMA5,SMA20,EMA5,EMA20,Close_SMA5_Crossover,Close_SMA5_Crossover_Direction,Days_Since_Close_SMA5_Crossover,Close_SMA20_Crossover,Close_SMA20_Crossover_Direction,Days_Since_Close_SMA20_Crossover,SMA5_SMA20_Crossover,SMA5_SMA20_Crossover_Direction,Days_Since_SMA5_SMA20_Crossover,Close_EMA5_Crossover,Close_EMA5_Crossover_Direction,Days_Since_Close_EMA5_Crossover,Close_EMA20_Crossover,Close_EMA20_Crossover_Direction,Days_Since_Close_EMA20_Crossover,EMA5_EMA20_Crossover,EMA5_EMA20_Crossover_Direction,Days_Since_EMA5_EMA20_Crossover,SMA5_EMA5_Crossover,SMA5_EMA5_Crossover_Direction,Days_Since_SMA5_EMA5_Crossover,SMA5_EMA20_Crossover,SMA5_EMA20_Crossover_Direction,Days_Since_SMA5_EMA20_Crossover,SMA20_EMA5_Crossover,SMA20_EMA5_Crossover_Direction,Days_Since_SMA20_EMA5_Crossover,SMA20_EMA20_Crossover,SMA20_EMA20_Crossover_Direction,Days_Since_SMA20_EMA20_Crossover
count,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000,5727.000000
mean,3.219415,3.251986,3.187073,3.219113,1143.059193,3.215317,3.201023,3.219889,3.222993,0.473721,0.000000,1.989873,0.475816,0.000175,5.685874,0.475642,0.000175,7.760608,0.470229,0.000000,2.260171,0.473721,0.000000,5.898202,0.471975,0.000000,10.514580,0.506897,0.000000,1.434433,0.473197,0.000175,9.951982,0.520866,-0.000175,7.260695,0.515977,0.000175,5.177231
std,1.314306,1.318372,1.310058,1.313802,13327.910160,1.313893,1.314589,1.313598,1.314004,0.499353,0.514546,2.882960,0.499458,0.350390,8.839449,0.499450,0.244749,12.444391,0.499156,0.513526,3.492078,0.499353,0.366707,9.658396,0.499258,0.242959,16.958871,0.499996,0.512505,1.965078,0.499325,0.235290,16.222894,0.499608,0.244035,11.911302,0.499788,0.253856,7.772403
min,0.484000,0.538000,0.398000,0.499000,0.000000,0.000000,0.000000,0.545452,0.588054,0.000000,-1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,-1.000000,0.000000
25%,2.161500,2.194000,2.135000,2.159000,0.000000,2.165400,2.154825,2.165694,2.164321,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.077000,3.102000,3.054000,3.072000,0.000000,3.061800,3.028300,3.064463,3.064818,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,4.228500,4.263000,4.192000,4.227000,0.000000,4.222500,4.211325,4.225602,4.220103,1.000000,0.000000,3.000000,1.000000,0.000000,9.000000,1.000000,0.000000,12.000000,1.000000,0.000000,3.000000,1.000000,0.000000,8.000000,1.000000,0.000000,15.000000,1.000000,0.000000,2.000000,1.000000,0.000000,14.000000,1.000000,0.000000,10.000000,1.000000,0.000000,9.000000
max,6.781000,6.823000,6.767000,6.781000,159700.000000,6.737200,6.657200,6.734979,6.679908,1.000000,1.000000,19.000000,1.000000,1.000000,51.000000,1.000000,1.000000,98.000000,1.000000,1.000000,32.000000,1.000000,1.000000,67.000000,1.000000,1.000000,100.000000,1.000000,1.000000,13.000000,1.000000,1.000000,100.000000,1.000000,1.000000,63.000000,1.000000,1.000000,50.000000
